In [2]:
import fitz
import re

In [3]:
doc = fitz.open('H422 X-PRESS MULHACEN 21.3.2022.pdf')

In [26]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 500)
base_triangle = (33, 100, 547, 580)

In [27]:
total_starts = []   # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
rect_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad
words_in_rectangles = [] # lista för ord inom rektang*larna, utan "headers"
total_pods = [] # lista över rektanglar och namn för PODs
list_of_base_triangles = []
list_of_rectangles = []
list_of_tod_rectangles = []
final_tod_list = []

for page in doc:

    # starts
    starts = page.search_for("Shipper :")
    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # lägger till på höjden för y0 och y1

    # stops
    stops = page.search_for("Units :")
    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height))

    # total_words
    words_in_page = page.get_text("words")
    for words in words_in_page:
        total_words.append([words[0], words[1] + total_height, words[2], words[3] + total_height, words[4]])

    # rect_pods
    pods = page.search_for('Port of discharge')
    for rect in pods:      
        rect_pods.append(rect + (0, total_height + 12.20998, 0, total_height + 16.08598)) # lägger till på y0 och y1 för att hitta POD
    
    # words_in_rectangles
    base_triangle = (33, 100 + total_height, 547, 580 + total_height) # Utgår från basrektangel (33, 100, 547, 580) och lägger på totalhöjden
    words_in_rectangles += [word for word in total_words if fitz.Rect(word[:4]).intersects(base_triangle)] # Tar fram alla ord inom rektangeln ovan
    list_of_base_triangles.append(base_triangle)
    
    total_height += page.rect.height # lägger till totalhöjden varje loop
       

pod_names = [' '.join([words[4] for words in total_words if fitz.Rect(words[:4]).intersects(pod)]) for pod in rect_pods] # Tar fram alla POD is en lista

rectangle = lambda x: fitz.Rect(33, total_starts[x][1], 580, total_stops[x][3])

for num, starts in enumerate(total_starts):
    list_of_rectangles.append(rectangle(num))

for triangles, tod in zip(list_of_base_triangles, pod_names):
    list_of_tod_rectangles.append([triangles[0], triangles[1], triangles[2], triangles[3], tod])

for rect in list_of_rectangles:
    counter = 0
    for tod in list_of_tod_rectangles:
        if tod[3] // rect[3] > 0 and counter == 0:
            final_tod_list.append(tod[4])
            counter == 1
            break

In [28]:
def gather_data(start, stop):
    rektangel = (464, start[1], 555, stop[3]) # rektangel för området
    samma_rad = start[1]        # värdet uppdateras varje loop med värdet från föregående loop
    lista_gathered_data = []    # varje ny rad representerar en instans i listan
    ordlista = ""
    
    for word in total_words:
            
        if fitz.Rect(word[:4]).intersects(rektangel):
            
            if samma_rad != word[1]:
                lista_gathered_data.append(ordlista.strip())
                ordlista = ""
                ordlista = word[4]

            elif samma_rad == word[1]:
                ordlista += " " + word[4]

            samma_rad = word[1]

    lista_gathered_data.append(ordlista.strip()) # lägger till den sista raden i loopen
       
    return lista_gathered_data


In [68]:
def get_container_nos(unit, start, stop):

    rectangle = (unit[0], start[1], unit[1], stop[3])
    same_row = start[1]
    list_of_data = []
    list_of_cont = []
    str_of_words = ""

    for word in words_in_rectangles:
        if fitz.Rect(word[:4]).intersects(rectangle):
            if same_row == word[1]:
                str_of_words += " " + word[4]

            elif same_row != word[1]:
                list_of_data.append(str_of_words.strip())
                str_of_words = word[4]

            same_row = word[1]

    for str in list_of_data:

        #matching CONTAINER
        match_container = re.search(r'^\w{4}\s\d{6}\-\d', str)
        trim_cont = re.sub(r'[\s-]', '', str)

        if match_container:
            list_of_cont.append(trim_cont)

    return list_of_cont

In [69]:
get_container_nos(unit, total_starts[0], total_stops[0])

['MSDU2319070', 'TRHU1290156']

In [63]:
unit_test1 = [
    'TRLU 894306-0',
    'MSDU 123571-4',
    'MSDU 208399-8',
    'TCKU 184381-8',
    'MEDU 525427-4'
    ]

unit_test2 = [
    'MSDU 231907-0',
    'Seal AE369005',
    'TRHU 129015-6',
    'Seal AE369004'
    ]

In [66]:
list_of_cont = []

for str in unit_test2:

    #matching CONTAINER
    match_container = re.search(r'^\w{4}\s\d{6}\-\d', str)
    trim_cont = re.sub(r'[\s-]', '', str)

    if match_container:
        list_of_cont.append(trim_cont)



['MSDU2319070', 'TRHU1290156']

In [ ]:
info = [
    223.44000244140625,
    99.7957534790039,
    247.55136108398438,
    111.04881286621094,
    'MSDU'
    ]

info2 = [
    223.44000244140625,
    128.4557647705078,
    246.18362426757812,
    139.7088165283203,
    'TRHU'
    ]

info3 = [
    248.46044921875,
    128.4557647705078,
    283.063232421875,
    139.7088165283203,
    '129015-6'
    ]

In [91]:
def get_goods_info(goods, start, stop):

    rectangle = (goods[0], start[1], goods[1], stop[3])
    samma_rad = start
    lista_container_data = []
    lista_ord = ""

    for word in words_in_rectangles:

        if fitz.Rect(word[:4]).intersects(rectangle):

            if samma_rad == word[1]:                                # Om y-värdet på föregående rad i loop är samma som y-värdet för denna runda
                lista_ord += " " + word[4]

            elif samma_rad != word[1]:                              # Om y-värdet inte stämmer överens
                lista_container_data.append(lista_ord.strip())
                lista_ord = word[4]

            samma_rad = word[1]                                     # Sparar Y-värdet för raden för denna loop. Används i nästa runda

    lista_container_data.remove('')                                  # Tar bort första "" i listan
    
    customs_found = [id for id, value in enumerate(lista_container_data) if re.search(r'^Customs', value)]  # Söker efter "Customs" i lista_container_data och letar match
    
    lista_manifest_data = lista_container_data[customs_found[0]:]                                         # Ny lista som kopierar info från match med "Customs" och till slutet
    lista_container_data = lista_container_data[:customs_found[0]]                                            # Gör om lista och tar info fram till "Customs" match

    return lista_container_data, lista_manifest_data



In [92]:
get_goods_info(goods, total_starts[3], total_stops[3])

(["20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL'],
 ['Customs status "N"',
  'Transhipment by MSC SORAYA Voy',
  'NM213A',
  'ETD 5.4.2022 ISTANBUL 13.4.2022',
  'Ref. 11221094469 OPS',
  '47499/1649789',
  'FREIGHT PAYABLE AT As per C/P'])

In [93]:
new_list2 = [
  'Customs status "N"',
  'Transhipment by MSC PETRA',
  'Voy. NT202A',
  'ETD 25.1.2022 GEBZE ETA',
  '1.2.2022',
  'Ref. 11221059091 OPS',
  '43914/1631983',
  'FREIGHT PAYABLE AT As per C/P'
  ]

new_list3 = [
  'Customs status "N"',
  'Transhipment by MSC CATERINA',
  'Voy. NA202A',
  'ETD 23.1.2022 NAVEGANTES, SC',
  'ETA 12.2.2022',
  'Ref. 79096203 OPS',
  '44089/1632869',
  'FREIGHT PAYABLE AT As per C/P'
  ]

new_list4 = [
  'Customs status "C"',
  'ref: EMPTY CONTAINER'
  ]

new_list5 = [
  'Customs status "N"',
  'Transhipment by MSC SORAYA Voy',
  'NM213A',
  'ETD 5.4.2022 ISTANBUL 13.4.2022',
  'Ref. 11221094469 OPS',
  '47499/1649789',
  'FREIGHT PAYABLE AT As per C/P'
  ]

new_list6 = [
  'Customs status "C"',
  '01706213/01706216',
  'FREIGHT PAYABLE AT As per C/P'
  ]

new_list7 = [
  'Customs status "N"',
  'Transhipment by THALASSA',
  'HELLAS Voy 0589-045E',
  'ETD 3.4.2022 SHANGHAI,',
  "PEOPLE'S REPUBLIC OF CHINA",
  '15.5.2022',
  'Ref. : 501200006755 OPS',
  '47664/1650646',
  'FREIGHT PAYABLE AT As per C/P',
  'MRN: 22FI000000282186E7'
  ]

new_list8 = [
  'Customs status "N"',
  'Transhipment by THALASSA',
  'HELLAS Voy 0589-045E',
  'ETD 3.4.2022',
  "SHANGHAI,PEOPLE'S REPUBLIC",
  'OF CHINA 20.5.2022',
  'Ref. : 501200006763 OPS',
  '47706/1650931',
  'FREIGHT PAYABLE AT As per C/P',
  'MRN: 22FI000000277461E0'
  ]

new_list9 = [
  'Customs status "N"',
  'Transhipment by EVER GLOBE Voy',
  '1176-011E',
  'ETD 31.3.2022 KAOHSIUNG,',
  'TAIWAN 4.5.2022',
  'Ref. : 501200006772 OPS',
  '47611/1650411',
  'FREIGHT PAYABLE AT As per C/P',
  'MRN: 22FI000000277321E5'
  ]

new_list10 = [
  'Customs status "C"',
  'Transhipment by MAERSK KOTKA',
  'Voy 213S',
  'ETD 5.4.2022 GEBZE 18.4..2022',
  'Ref. 2FIN000050 OPS',
  'FERROCHROME/Jukka Knuuti',
  'FREIGHT PAYABLE AT As per C/P',
  'MRN: 22FI000000282437E5'
  ]

new_list11 = [
  'Customs status "N"',
  'Transhipment by MSC SORAYA Voy',
  'NM213A',
  'ETD 5.4.2022 ISTANBUL 13.4.2022',
  'Ref. 11221094469 OPS',
  '47499/1649789',
  'FREIGHT PAYABLE AT As per C/P'
  ]

new_list12 = [
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL',
  "20' DC",
  '2 PKG STAINLESS STEEL'
  ]

In [77]:
ocean_vessel_list = []
final_pod_list = []
counter_pod = 0
counter_booking = 0

for val in new_list11:

    #matching CUSTOMS STATUS
    customs_match = re.search(r'(?<=Customs status \")\w', val)
    if customs_match:
        customs_status = customs_match.group()

    #matching OCEAN VESSEL
    transhipment_match = re.search(r'^Transhipment by', val)
    voy_match = re.search(r'^.+(?= Voy)', val)
    vessel_match = re.search(r'(?<=Transhipment by\s).+[^\s*Voy]', val)
    vessel_match_rest = re.search(r'^\w+[^ Voy]', val)

    if transhipment_match and vessel_match and voy_match:
        ocean_vessel_list.append(vessel_match.group())

    elif transhipment_match and not voy_match:
        ocean_vessel_list.append(vessel_match.group())

    elif voy_match and not transhipment_match:
        ocean_vessel_list.append(vessel_match_rest.group())
    
    ocean_vessel = ' '.join(ocean_vessel_list)

    #matching FINAL POD
    date_last_match = re.search(r'\d+\.+\d+\.+\d+$', val)
    etd_match = re.search(r'^ETD\s*\d+\.+\d+\.+\d+\s*\D+', val)
    only_etd_date = re.search(r'^(ETD\s*\d+\.+\d+\.+\d+)$', val)
    fpod_dual_dates = re.search(r'^(ETD\s*\d+\.+\d+\.+\d+\s*)(\D*)(\s+\d+\.+\d+\.+\d+)', val)
    fpod_single_date = re.search(r'^(ETD\s*\d+\.+\d+\.+\d+\s*)(\D*)', val)
    get_all_upper = re.search(r'^[\D]*', val)

    def del_fillers_pod(string):
        new_string = re.sub(r'\s*ETA\s*', '', string)
        new_string2 = re.sub(r',[^,]*', '', new_string)
        new_string3 = re.sub(r'\s$', '', new_string2)
        return new_string3

    if etd_match and date_last_match:
        final_pod_list.append(del_fillers_pod(fpod_dual_dates.group(2)))

    elif etd_match and not date_last_match:
        final_pod_list.append(del_fillers_pod(fpod_single_date.group(2)))

    elif counter_pod == 1:
        final_pod_list.append(del_fillers_pod(get_all_upper.group()))
        counter_pod = 0

    elif counter_pod == 2:
        final_pod_list.append(del_fillers_pod(get_all_upper.group()))
        counter_pod == 0

    elif only_etd_date:
        counter_pod = 1

    elif etd_match and not date_last_match:
        counter_pod = 2

    final_pod = ' '.join(final_pod_list)

    #matching BOOKING NUMBER
    ref_match = re.search(r'^ref', val, flags=re.IGNORECASE)

    def del_fillers_ref(string):
        new_string = re.sub(r'^[RrEeFf]{3}[:\.\s]*', '', string)
        new_string2 = re.sub(r'\s*OPS', '', new_string)
        return new_string2

    if ref_match and counter_booking != 1:
        booking_number = del_fillers_ref(val)

    elif counter_booking == 1:
        if transhipment_match:
            counter_booking = 0

        if not transhipment_match:
            booking_number = del_fillers_ref(val)
            counter_booking = 0

    elif customs_match:
        counter_booking = 1


    
print("1:", customs_status, " 2:", ocean_vessel, " 3:", final_pod, " 4:", booking_number)



1: N  2: MSC SORAYA  3: ISTANBUL  4: 11221094469


In [103]:
list_of_unit_types = []
list_of_goods = []

for val in new_list12:

    match_unit_type = re.search(r"^\d{2}\'\D+", val)
    match_goods_info = re.search(r'^(\d+)\s+\w*\s+(\D+)', val)
    match_goods_info2 = re.search(r'^\D*$', val)


    if match_unit_type:
        list_of_unit_types.append(match_unit_type.group())

    if match_goods_info:
        list_of_goods.append(match_goods_info.group(1, 2))
    elif match_goods_info2:
        list_of_goods.append(match_goods_info2.group())

print(list_of_unit_types, list_of_goods)

["20' DC", "20' DC", "20' DC", "20' DC", "20' DC"] [('2', 'STAINLESS STEEL'), ('2', 'STAINLESS STEEL'), ('2', 'STAINLESS STEEL'), ('2', 'STAINLESS STEEL'), ('2', 'STAINLESS STEEL')]


'20 DC'

In [ ]:
def breakdown_ytterligare_info():

    ny_lista = new_list2

    customs_status = ""
    ocean_vessel = ""
    voyage = ""
    final_dest = ""

    for val in ny_lista:

        if re.search(r'^Customs\s.{6}\s\".\"', val, i):
            customs_status = val[16]

        if re.search(r'Transhipment\sby\s', val, i):
            ocean_vessel = val[16:]

        if re.search(r'Voy.', val, i):      # Ej klar
            voyage = val[4:]

        #if re.search(r'', val, i):
        #    final_dest = ""

    return customs_status, ocean_vessel, voyage

breakdown_ytterligare_info()

In [ ]:
test_list = [1, 4, 5, 6, 4, 5, 6, 5, 4]

size = len(test_list)
idx_list = [idx + 1 for idx, val in
            enumerate(test_list) if val == 5]

new_list = []
for i,j in zip([0, 3, 6, 8], [3, 6, 8, 9]):
    new_list.append(test_list[i: j])

print(new_list)